<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/TensorFlow2/Tutorial/Neural_translation_English_German.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project
## Neural translation model
### Instructions

In this notebook, you will create a neural network that translates from English to German. You will use concepts from throughout this course, including building more flexible model architectures, freezing layers, data processing pipeline and sequence modelling.

This project is peer-assessed. Within this notebook you will find instructions in each section for how to complete the project. Pay close attention to the instructions as the peer review will be carried out according to a grading rubric that checks key parts of the project instructions. Feel free to add extra cells into the notebook as required.

### How to submit

When you have completed the Capstone project notebook, you will submit a pdf of the notebook for peer review. First ensure that the notebook has been fully executed from beginning to end, and all of the cell outputs are visible. This is important, as the grading rubric depends on the reviewer being able to view the outputs of your notebook. Save the notebook as a pdf (you could download the notebook with File -> Download .ipynb, open the notebook locally, and then File -> Download as -> PDF via LaTeX), and then submit this pdf for review.

### Let's get started!

We'll start by running some imports, and loading the dataset. For this project you are free to make further imports throughout the notebook as you wish. 

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import unicodedata
import re
from IPython.display import Image
import os
import json
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Layer

![Flags overview image](https://github.com/MengOonLee/Deep_learning/blob/master/TensorFlow2/Tutorial/image/neural_translation/germany_uk_flags.png?raw=1)

For the capstone project, you will use a language dataset from http://www.manythings.org/anki/ to build a neural translation model. This dataset consists of over 200,000 pairs of sentences in English and German. In order to make the training quicker, we will restrict to our dataset to 20,000 pairs. Feel free to change this if you wish - the size of the dataset used is not part of the grading rubric.

Your goal is to develop a neural translation model from English to German, making use of a pre-trained English word embedding module.

#### Import the data

The dataset is available for download as a zip file at the following link:

https://drive.google.com/open?id=1KczOciG7sYY7SB9UlBeRP1T9659b121Q

You should store the unzipped folder in local drive for use in this Colab notebook.

In [2]:
if not os.path.exists("./data"):
    os.makedirs("./data")

In [3]:
%%bash

fileid="1KczOciG7sYY7SB9UlBeRP1T9659b121Q"
filename="./data/deu.txt"
curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=${fileid}" > /dev/null
curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=${fileid}" -o ${filename}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    213      0 --:--:--  0:00:01 --:--:--   213
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100 29.9M    0 29.9M    0     0  12.4M      0 --:--:--  0:00:02 --:--:-- 12.4M


In [4]:
# Run this cell to load the dataset

NUM_EXAMPLES = 20000
data_examples = []
with open('data/deu.txt', 'r', encoding='utf8') as f:
    for line in f.readlines():
        if len(data_examples) < NUM_EXAMPLES:
            data_examples.append(line)
        else:
            break

In [5]:
# These functions preprocess English and German sentences

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) 
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"ü", 'ue', sentence)
    sentence = re.sub(r"ä", 'ae', sentence)
    sentence = re.sub(r"ö", 'oe', sentence)
    sentence = re.sub(r'ß', 'ss', sentence)
    
    sentence = unicode_to_ascii(sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-z?.!,']+", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    return sentence.strip()

#### The custom translation model
The following is a schematic of the custom translation model architecture you will develop in this project.

![Model Schematic](https://github.com/MengOonLee/Deep_learning/blob/master/TensorFlow2/Tutorial/image/neural_translation/neural_translation_model.png?raw=1)

Key:
![Model key](https://github.com/MengOonLee/Deep_learning/blob/master/TensorFlow2/Tutorial/image/neural_translation/neural_translation_model_key.png?raw=1)

The custom model consists of an encoder RNN and a decoder RNN. The encoder takes words of an English sentence as input, and uses a pre-trained word embedding to embed the words into a 128-dimensional space. To indicate the end of the input sentence, a special end token (in the same 128-dimensional space) is passed in as an input. This token is a TensorFlow Variable that is learned in the training phase (unlike the pre-trained word embedding, which is frozen).

The decoder RNN takes the internal state of the encoder network as its initial state. A start token is passed in as the first input, which is embedded using a learned German word embedding. The decoder RNN then makes a prediction for the next German word, which during inference is then passed in as the following input, and this process is repeated until the special `<end>` token is emitted from the decoder.

## 1. Text preprocessing
* Create separate lists of English and German sentences, and preprocess them using the `preprocess_sentence` function provided for you above.
* Add a special `"<start>"` and `"<end>"` token to the beginning and end of every German sentence.
* Use the Tokenizer class from the `tf.keras.preprocessing.text` module to tokenize the German sentences, ensuring that no character filters are applied. _Hint: use the Tokenizer's "filter" keyword argument._
* Print out at least 5 randomly chosen examples of (preprocessed) English and German sentence pairs. For the German sentence, print out the text (with start and end tokens) as well as the tokenized sequence.
* Pad the end of the tokenized German sequences with zeros, and batch the complete set of sequences into one numpy array.

In [6]:
def create_sentence(data):
    """
    Clean the input and return sentence pairs.
    """
    sentence_pairs = [[preprocess_sentence(w) for w in l.split('\t')][:2] 
        for l in data]

    return zip(*sentence_pairs)

# get pairs
en_sentence, ge_sentence = create_sentence(data_examples)
# show last 5 pairs
for pair in zip(en_sentence[-5:], ge_sentence[-5:]):
    print(pair)

("i've been working .", 'ich war arbeiten .')
("i've bought a car .", 'ich habe ein auto gekauft .')
("i've come for tom .", 'ich bin fuer tom gekommen .')
("i've done nothing .", 'ich habe nichts getan .')
("i've eaten enough .", 'ich habe genug gegessen .')


In [7]:
def add_start_end(sentences):
    """
    Adding a start and end token to the sentence so that the
    model know when to start and stop predicting.
    """
    
    return ["<start> " + s + " <end>" for s in sentences]

ge_sentence = add_start_end(ge_sentence)
ge_sentence[-5:]

['<start> ich war arbeiten . <end>',
 '<start> ich habe ein auto gekauft . <end>',
 '<start> ich bin fuer tom gekommen . <end>',
 '<start> ich habe nichts getan . <end>',
 '<start> ich habe genug gegessen . <end>']

In [8]:
def create_tokenizer(sentences):
    """
    Create and return a Tokenizer.
    """
    tokenizer = Tokenizer(filters='')
    tokenizer.fit_on_texts(sentences)

    return tokenizer

# Build the German tokenizer vocabulary
ge_tokenizer = create_tokenizer(ge_sentence)
# Tokenize the German sentences
ge_sequence = ge_tokenizer.texts_to_sequences(ge_sentence)
ge_sequence[-5:]

[[1, 4, 24, 252, 3, 2],
 [1, 4, 18, 19, 82, 280, 3, 2],
 [1, 4, 15, 77, 5, 276, 3, 2],
 [1, 4, 18, 108, 182, 3, 2],
 [1, 4, 18, 480, 217, 3, 2]]

In [9]:
# Randomly select 5 examples of English and German sentence pairs
inx = np.random.choice(len(en_sentence), 5, replace=False)

for n, i in enumerate(inx):
    print(n, en_sentence[i])
    print(n, ge_sentence[i], ge_sequence[i])

0 it's stuck .
0 <start> es steckt fest . <end> [1, 10, 2334, 735, 3, 2]
1 it sure is hot .
1 <start> es ist aber ganz schoen heiss . <end> [1, 10, 6, 897, 194, 322, 187, 3, 2]
2 i couldn't sleep .
2 <start> ich konnte nicht schlafen . <end> [1, 4, 441, 12, 260, 3, 2]
3 do you follow ?
3 <start> kannst du mir folgen ? <end> [1, 109, 13, 21, 492, 7, 2]
4 i was in all day .
4 <start> ich war den ganzen tag zu hause . <end> [1, 4, 24, 53, 1427, 214, 20, 75, 3, 2]


In [10]:
def post_pad(sequences):
    """
    Pad each sequence to a maximum length.
    """

    return pad_sequences(sequences, padding='post')

ge_padded = post_pad(ge_sequence)
ge_padded[-5:]

array([[  1,   4,  24, 252,   3,   2,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  1,   4,  18,  19,  82, 280,   3,   2,   0,   0,   0,   0,   0,
          0],
       [  1,   4,  15,  77,   5, 276,   3,   2,   0,   0,   0,   0,   0,
          0],
       [  1,   4,  18, 108, 182,   3,   2,   0,   0,   0,   0,   0,   0,
          0],
       [  1,   4,  18, 480, 217,   3,   2,   0,   0,   0,   0,   0,   0,
          0]], dtype=int32)

## 2. Prepare the data with tf.data.Dataset objects

#### Load the embedding layer
As part of the dataset preproceessing for this project, you will use a pre-trained English word embedding module from TensorFlow Hub. The URL for the module is https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1.

This embedding takes a batch of text tokens in a 1-D tensor of strings as input. It then embeds the separate tokens into a 128-dimensional space. 

The code to load and test the embedding layer is provided for you below.

**NB:** this model can also be used as a sentence embedding module. The module will process each token by removing punctuation and splitting on spaces. It then averages the word embeddings over a sentence to give a single embedding vector. However, we will use it only as a word embedding module, and will pass each word in the input sentence as a separate token.

In [11]:
# Load embedding module from Tensorflow Hub

embedding_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1", 
    output_shape=[128], input_shape=[], dtype=tf.string)

In [12]:
# Test the layer

embedding_layer(tf.constant(["these", "aren't", "the", "droids", "you're", "looking", "for"])).shape

TensorShape([7, 128])

You should now prepare the training and validation Datasets.

* Create a random training and validation set split of the data, reserving e.g. 20% of the data for validation (NB: each English dataset example is a single sentence string, and each German dataset example is a sequence of padded integer tokens).
* Load the training and validation sets into a tf.data.Dataset object, passing in a tuple of English and German data for both training and validation sets.
* Create a function to map over the datasets that splits each English sentence at spaces. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.strings.split function._
* Create a function to map over the datasets that embeds each sequence of English words using the loaded embedding layer/model. Apply this function to both Dataset objects using the map method.
* Create a function to filter out dataset examples where the English sentence is greater than or equal to than 13 (embedded) tokens in length. Apply this function to both Dataset objects using the filter method.
* Create a function to map over the datasets that pads each English sequence of embeddings with some distinct padding value before the sequence, so that each sequence is length 13. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.pad function. You can extract a Tensor shape using tf.shape; you might also find the tf.math.maximum function useful._
* Batch both training and validation Datasets with a batch size of 16.
* Print the `element_spec` property for the training and validation Datasets. 
* Using the Dataset `.take(1)` method, print the shape of the English data example from the training Dataset.
* Using the Dataset `.take(1)` method, print the German data example Tensor from the validation Dataset.

In [13]:
# Creating training and validation sets using an 80-20 split
en_train, en_val, ge_train, ge_val = train_test_split(
    en_sentence, ge_padded, test_size=0.2)

# Show length
print(len(en_train), len(ge_train), len(en_val), len(ge_val))

16000 16000 4000 4000


In [14]:
def create_dataset(input, target):
    """
    Takes an input language, and a corresponding target language 
    to create a tf.data.Dataset object with these input and target.
    """

    return tf.data.Dataset.from_tensor_slices((input, target))

# Create Dataset object for both training and vailidation sets
train_dataset = create_dataset(en_train, ge_train)
valid_dataset = create_dataset(en_val, ge_val)

# Print the first 5 elements of the dataset
for elem in train_dataset.take(5):
    print(elem[0].numpy(), elem[1].numpy())

train_dataset.element_spec

b'i want a dog .' [  1   4  66  40 107  35   3   2   0   0   0   0   0   0]
b'have a look .' [   1 1470   76    3    2    0    0    0    0    0    0    0    0    0]
b'i have your key .' [  1   4  18 334 204   3   2   0   0   0   0   0   0   0]
b'am i a bad person ?' [  1  15   4  19 539 517   7   2   0   0   0   0   0   0]
b'tom told mary .' [  1   5 241  10 119   3   2   0   0   0   0   0   0   0]


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(14,), dtype=tf.int32, name=None))

In [15]:
def map_split(x, _):
    """
    Splits each English sentence at spaces.
    """

    return (tf.strings.split(x, sep=' '), _)

# Map the function over the both Dataset objects. 
train_dataset = train_dataset.map(map_split)
valid_dataset = valid_dataset.map(map_split)

# Print the first 5 elements of the dataset
for elem in train_dataset.take(5):
    print(elem[0].numpy(), elem[1].numpy())

train_dataset.element_spec

[b'i' b'want' b'a' b'dog' b'.'] [  1   4  66  40 107  35   3   2   0   0   0   0   0   0]
[b'have' b'a' b'look' b'.'] [   1 1470   76    3    2    0    0    0    0    0    0    0    0    0]
[b'i' b'have' b'your' b'key' b'.'] [  1   4  18 334 204   3   2   0   0   0   0   0   0   0]
[b'am' b'i' b'a' b'bad' b'person' b'?'] [  1  15   4  19 539 517   7   2   0   0   0   0   0   0]
[b'tom' b'told' b'mary' b'.'] [  1   5 241  10 119   3   2   0   0   0   0   0   0   0]


(TensorSpec(shape=(None,), dtype=tf.string, name=None),
 TensorSpec(shape=(14,), dtype=tf.int32, name=None))

In [16]:
def map_embed(x, _):
    """
    Embeds each sequence of English words using the loaded embedding layer.
    """

    return (embedding_layer(x), _)

# Map the function over the both Dataset objects. 
train_dataset = train_dataset.map(map_embed)
valid_dataset = valid_dataset.map(map_embed)

# Print the shape of first 5 elements of the dataset
for elem in train_dataset.take(5):
    print(f"Input shape: {elem[0].shape}, Target shape: {elem[1].shape}")

train_dataset.element_spec

Input shape: (5, 128), Target shape: (14,)
Input shape: (4, 128), Target shape: (14,)
Input shape: (5, 128), Target shape: (14,)
Input shape: (6, 128), Target shape: (14,)
Input shape: (4, 128), Target shape: (14,)


(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None),
 TensorSpec(shape=(14,), dtype=tf.int32, name=None))

In [17]:
def filter_length(x, _):
    """
    Filter out dataset examples where the English sentence is 
    greater than or equal to 13 (embedded) tokens in length.
    """

    return len(x)<13

# Filter over the both Dataset objects.
train_dataset = train_dataset.filter(filter_length)
valid_dataset = valid_dataset.filter(filter_length)

# Print the shape of first 5 elements of the dataset
for elem in train_dataset.take(5):
    print(f"Input tokens: {len(elem[0])}, Target tokens: {len(elem[1])}")

train_dataset.element_spec

Input tokens: 5, Target tokens: 14
Input tokens: 4, Target tokens: 14
Input tokens: 5, Target tokens: 14
Input tokens: 6, Target tokens: 14
Input tokens: 4, Target tokens: 14


(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None),
 TensorSpec(shape=(14,), dtype=tf.int32, name=None))

In [18]:
def map_pad_embedding(x, _):
    """
    Pads each English sequence of embeddings with some distinct
    padding value before the sequence, so that each sequence is
    length 13.
    """
    pad_size = tf.math.maximum(0, 13 - tf.shape(x)[0])
    paddings = tf.concat(([[pad_size, 0]], [[0, 0]]), axis=0)
    padded = tf.pad(x, paddings, "CONSTANT")

    return (tf.reshape(padded, [13, 128]), _)

# Map the function over the both Dataset objects. 
train_dataset = train_dataset.map(map_pad_embedding)
valid_dataset = valid_dataset.map(map_pad_embedding)

# Print the shape of first 5 elements of the dataset
for elem in train_dataset.take(5):
    print(f"Input shape: {elem[0].shape}, Target shape: {elem[1].shape}")

Input shape: (13, 128), Target shape: (14,)
Input shape: (13, 128), Target shape: (14,)
Input shape: (13, 128), Target shape: (14,)
Input shape: (13, 128), Target shape: (14,)
Input shape: (13, 128), Target shape: (14,)


In [19]:
def create_batch(dataset, batch_size):
  """
  Batch the dataset using the batch_size argument, 
  setting drop_remainder to True.
  """

  return dataset.batch(batch_size, drop_remainder=True)

# Batch the both datasets with a batch size of 16.
train_dataset = create_batch(train_dataset, 16)
valid_dataset = create_batch(valid_dataset, 16)

# Print the element_spec property for both datasets
print("The training dataset's element_spec property:\n")
print(train_dataset.element_spec, "\n")
print("The validation dataset's element_spec property:\n")
print(valid_dataset.element_spec)

The training dataset's element_spec property:

(TensorSpec(shape=(16, 13, 128), dtype=tf.float32, name=None), TensorSpec(shape=(16, 14), dtype=tf.int32, name=None)) 

The validation dataset's element_spec property:

(TensorSpec(shape=(16, 13, 128), dtype=tf.float32, name=None), TensorSpec(shape=(16, 14), dtype=tf.int32, name=None))


In [20]:
# Print the shape of the English data example from the training Dataset
for elem in train_dataset.take(1):
    print(f"The shape of English data example: {tf.shape(elem[0])}")

# Print the German data example Tensor from the validation Dataset
for elem in valid_dataset.take(1):
    print(f"The German data example Tensor:\n {elem[1]}")

The shape of English data example: [ 16  13 128]
The German data example Tensor:
 [[   1    4   15  701    3    2    0    0    0    0    0    0    0    0]
 [   1   81    8   11  114    7    2    0    0    0    0    0    0    0]
 [   1    5   16   34 2375    3    2    0    0    0    0    0    0    0]
 [   1   13  109   21  408    3    2    0    0    0    0    0    0    0]
 [   1   17   23 1634    3    2    0    0    0    0    0    0    0    0]
 [   1   43    6    8    7    2    0    0    0    0    0    0    0    0]
 [   1    4   15   56  167    3    2    0    0    0    0    0    0    0]
 [   1    4  518   12   20    3    2    0    0    0    0    0    0    0]
 [   1   81   17    5   12 5438    7    2    0    0    0    0    0    0]
 [   1   43   16 2976 2165    7    2    0    0    0    0    0    0    0]
 [   1    5    6  104 5208    3    2    0    0    0    0    0    0    0]
 [   1    5   16 2414    3    2    0    0    0    0    0    0    0    0]
 [   1    5   16  200    3    2    0    0 

## 3. Create the custom layer
You will now create a custom layer to add the learned end token embedding to the encoder model:

![Encoder schematic](https://github.com/MengOonLee/Deep_learning/blob/master/TensorFlow2/Tutorial/image/neural_translation/neural_translation_model_encoder.png?raw=1)

You should now build the custom layer.
* Using layer subclassing, create a custom layer that takes a batch of English data examples from one of the Datasets, and adds a learned embedded ‘end’ token to the end of each sequence. 
* This layer should create a TensorFlow Variable (that will be learned during training) that is 128-dimensional (the size of the embedding space). _Hint: you may find it helpful in the call method to use the tf.tile function to replicate the end token embedding across every element in the batch._
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and print the shape. Test the custom layer by calling the layer on the English data batch Tensor and print the resulting Tensor shape (the layer should increase the sequence length by one).

In [30]:
# x = tf.Variable("end")[np.newaxis]
x = tf.Variable(initial_value='end')[np.newaxis]
# tf.tile(x, [16])
embedding_layer(x)

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 0.10629286, -0.04656535,  0.02235991, -0.0167439 , -0.02687744,
         0.11218218,  0.16737002,  0.03516154,  0.08201192, -0.14556195,
        -0.16737652, -0.02462974, -0.02016166,  0.09759618, -0.04789549,
        -0.04571156,  0.00210454, -0.04578184, -0.01971264,  0.00306896,
        -0.10278008, -0.19465092,  0.15711933, -0.04499053,  0.15860957,
         0.00394357, -0.05481301,  0.08361798, -0.2101636 , -0.03423096,
         0.04670201,  0.03936931, -0.04615017,  0.14950292, -0.03206915,
        -0.00536091,  0.15267208,  0.00036312,  0.00837651,  0.02410653,
         0.00359997,  0.04296277,  0.16075055,  0.07331654,  0.13419327,
         0.11792442, -0.06390144,  0.19219498,  0.0993454 , -0.00867325,
        -0.06327151,  0.04767163, -0.14419536,  0.03899188, -0.12005239,
        -0.04458055,  0.14097413,  0.05163342,  0.03932897, -0.01676602,
        -0.16452752,  0.05091499,  0.07613431,  0.01514044, -0.02295079,
 

In [ ]:
class EndToken(Layer):
    """
    A custom layer takes a batch of English Datasets, 
    and adds a learned embedded 'end' token to the end of each sequence.
    """

    def __init__(self, **kwargs):
        super(EndToken, self).__init__(**kwargs)
        self.end_token = tf.expand_dims(
            tf.Variable(initial_value='end'), axis=0)
      
    # def build(self, input_shape):
    #     self.end_token = tf.Variable(initial_value='end')

    def call(self, inputs):
      # s = inputs.map(lambda x, _: x)
      # replicate_end_token = tf.tile(self.end_token, [16, 1, 128])
      return self.end_token
      # inputs.map(lambda x, _: x + )
      # replicate = tf.tile(self.end_token, inputs.)
      # return tf.concat(inputs, replicate)

In [ ]:
end_token = EndToken()

end_token(train_dataset)

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'end'], dtype=object)>

In [ ]:
for elem in train_dataset.take(1):
    print(f"The shape for a batch of English data example: {tf.shape(elem[0])}")

The shape for a batch of English data example: [ 16  13 128]


## 4. Build the encoder network
The encoder network follows the schematic diagram above. You should now build the RNN encoder model.
* Using the functional API, build the encoder network according to the following spec:
    * The model will take a batch of sequences of embedded English words as input, as given by the Dataset objects.
    * The next layer in the encoder will be the custom layer you created previously, to add a learned end token embedding to the end of the English sequence.
    * This is followed by a Masking layer, with the `mask_value` set to the distinct padding value you used when you padded the English sequences with the Dataset preprocessing above.
    * The final layer is an LSTM layer with 512 units, which also returns the hidden and cell states.
    * The encoder is a multi-output model. There should be two output Tensors of this model: the hidden state and cell states of the LSTM layer. The output of the LSTM layer is unused.
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and test the encoder model by calling it on the English data Tensor, and print the shape of the resulting Tensor outputs.
* Print the model summary for the encoder network.

## 5. Build the decoder network
The decoder network follows the schematic diagram below.

![Decoder schematic](https://github.com/MengOonLee/Deep_learning/blob/master/TensorFlow2/Tutorial/image/neural_translation/neural_translation_model_decoder.png?raw=1)

You should now build the RNN decoder model.
* Using Model subclassing, build the decoder network according to the following spec:
    * The initializer should create the following layers:
        * An Embedding layer with vocabulary size set to the number of unique German tokens, embedding dimension 128, and set to mask zero values in the input.
        * An LSTM layer with 512 units, that returns its hidden and cell states, and also returns sequences.
        * A Dense layer with number of units equal to the number of unique German tokens, and no activation function.
    * The call method should include the usual `inputs` argument, as well as the additional keyword arguments `hidden_state` and `cell_state`. The default value for these keyword arguments should be `None`.
    * The call method should pass the inputs through the Embedding layer, and then through the LSTM layer. If the `hidden_state` and `cell_state` arguments are provided, these should be used for the initial state of the LSTM layer. _Hint: use the_ `initial_state` _keyword argument when calling the LSTM layer on its input._
    * The call method should pass the LSTM output sequence through the Dense layer, and return the resulting Tensor, along with the hidden and cell states of the LSTM layer.
* Using the Dataset `.take(1)` method, extract a batch of English and German data examples from the training Dataset. Test the decoder model by first calling the encoder model on the English data Tensor to get the hidden and cell states, and then call the decoder model on the German data Tensor and hidden and cell states, and print the shape of the resulting decoder Tensor outputs.
* Print the model summary for the decoder network.

## 6. Make a custom training loop
You should now write a custom training loop to train your custom neural translation model.
* Define a function that takes a Tensor batch of German data (as extracted from the training Dataset), and returns a tuple containing German inputs and outputs for the decoder model (refer to schematic diagram above).
* Define a function that computes the forward and backward pass for your translation model. This function should take an English input, German input and German output as arguments, and should do the following:
    * Pass the English input into the encoder, to get the hidden and cell states of the encoder LSTM.
    * These hidden and cell states are then passed into the decoder, along with the German inputs, which returns a sequence of outputs (the hidden and cell state outputs of the decoder LSTM are unused in this function).
    * The loss should then be computed between the decoder outputs and the German output function argument.
    * The function returns the loss and gradients with respect to the encoder and decoder’s trainable variables.
    * Decorate the function with `@tf.function`
* Define and run a custom training loop for a number of epochs (for you to choose) that does the following:
    * Iterates through the training dataset, and creates decoder inputs and outputs from the German sequences.
    * Updates the parameters of the translation model using the gradients of the function above and an optimizer object.
    * Every epoch, compute the validation loss on a number of batches from the validation and save the epoch training and validation losses.
* Plot the learning curves for loss vs epoch for both training and validation sets.

_Hint: This model is computationally demanding to train. The quality of the model or length of training is not a factor in the grading rubric. However, to obtain a better model we recommend using the GPU accelerator hardware on Colab._

## 7. Use the model to translate
Now it's time to put your model into practice! You should run your translation for five randomly sampled English sentences from the dataset. For each sentence, the process is as follows:
* Preprocess and embed the English sentence according to the model requirements.
* Pass the embedded sentence through the encoder to get the encoder hidden and cell states.
* Starting with the special  `"<start>"` token, use this token and the final encoder hidden and cell states to get the one-step prediction from the decoder, as well as the decoder’s updated hidden and cell states.
* Create a loop to get the next step prediction and updated hidden and cell states from the decoder, using the most recent hidden and cell states. Terminate the loop when the `"<end>"` token is emitted, or when the sentence has reached a maximum length.
* Decode the output token sequence into German text and print the English text and the model's German translation.